# IHDP dataset: Dragonnet vs Meta Learners

In [2]:
import os, sys

In [3]:
sys.path.append('../causalml/inference/nn')

In [4]:
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
import statsmodels.api as sm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import warnings

from causalml.inference.meta import LRSRegressor
from causalml.inference.meta import XGBTRegressor, MLPTRegressor
from causalml.inference.meta import BaseXRegressor, BaseRRegressor, BaseSRegressor, BaseTRegressor
from causalml.match import NearestNeighborMatch, MatchOptimizer, create_table_one
from causalml.propensity import ElasticNetPropensityModel
from causalml.dataset import *
from causalml.metrics import *

from dragonnet import train_dragon

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

%matplotlib inline

In [6]:
import causalml
print(causalml.__version__)

0.6.0


# Part A: IHDP dataset

In [31]:
ihdp_dir = '/Users/mike.yung/Documents/repos/dragonnet/dat/ihdp/csv'

In [32]:
sorted(os.listdir(ihdp_dir))

['ihdp_npci_1.csv',
 'ihdp_npci_10.csv',
 'ihdp_npci_11.csv',
 'ihdp_npci_12.csv',
 'ihdp_npci_13.csv',
 'ihdp_npci_14.csv',
 'ihdp_npci_15.csv',
 'ihdp_npci_16.csv',
 'ihdp_npci_17.csv',
 'ihdp_npci_18.csv',
 'ihdp_npci_19.csv',
 'ihdp_npci_2.csv',
 'ihdp_npci_20.csv',
 'ihdp_npci_21.csv',
 'ihdp_npci_22.csv',
 'ihdp_npci_23.csv',
 'ihdp_npci_24.csv',
 'ihdp_npci_25.csv',
 'ihdp_npci_26.csv',
 'ihdp_npci_27.csv',
 'ihdp_npci_28.csv',
 'ihdp_npci_29.csv',
 'ihdp_npci_3.csv',
 'ihdp_npci_30.csv',
 'ihdp_npci_31.csv',
 'ihdp_npci_32.csv',
 'ihdp_npci_33.csv',
 'ihdp_npci_34.csv',
 'ihdp_npci_35.csv',
 'ihdp_npci_36.csv',
 'ihdp_npci_37.csv',
 'ihdp_npci_38.csv',
 'ihdp_npci_39.csv',
 'ihdp_npci_4.csv',
 'ihdp_npci_40.csv',
 'ihdp_npci_41.csv',
 'ihdp_npci_42.csv',
 'ihdp_npci_43.csv',
 'ihdp_npci_44.csv',
 'ihdp_npci_45.csv',
 'ihdp_npci_46.csv',
 'ihdp_npci_47.csv',
 'ihdp_npci_48.csv',
 'ihdp_npci_49.csv',
 'ihdp_npci_5.csv',
 'ihdp_npci_50.csv',
 'ihdp_npci_6.csv',
 'ihdp_npci_7.csv',
 'ihdp_npci_8.csv',
 'ihdp_npci_9.csv']

In [479]:
df = pd.read_csv(f'{ihdp_dir}/ihdp_npci_1.csv', header=None)
col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ]

for i in range(1,26):
    col.append("x"+str(i))
df.columns = col

In [480]:
df.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,1,1,1,1,0,0,0,0,0,0


In [360]:
pd.Series(df['treatment']).value_counts(normalize=True)

0    0.813922
1    0.186078
Name: treatment, dtype: float64

In [361]:
X = df.loc[:,'x1':]
treatment = df['treatment']
y = df['y_factual']
tau = df.apply(lambda d: d['y_factual'] - d['y_cfactual']
               if d['treatment']==1
               else d['y_cfactual'] - d['y_factual'], axis=1)

In [362]:
p_model = LogisticRegressionCV(penalty='elasticnet', solver='saga', l1_ratios=np.linspace(0,1,10),
                               cv=StratifiedKFold(n_splits=2, shuffle=False))
p_model.fit(X, treatment)
p = p_model.predict_proba(X)[:, 1]

In [363]:
s_learner = BaseSRegressor(LGBMRegressor())
s_ate = s_learner.estimate_ate(X, treatment, y)[0]
s_ite = s_learner.fit_predict(X, treatment, y)

t_learner = BaseTRegressor(LGBMRegressor())
t_ate = t_learner.estimate_ate(X, treatment, y)[0][0]
t_ite = t_learner.fit_predict(X, treatment, y)

x_learner = BaseXRegressor(LGBMRegressor())
x_ate = x_learner.estimate_ate(X, p, treatment, y)[0][0]
x_ite = x_learner.fit_predict(X, p, treatment, y)

r_learner = BaseRRegressor(LGBMRegressor())
r_ate = r_learner.estimate_ate(X, p, treatment, y)[0][0]
r_ite = r_learner.fit_predict(X, p, treatment, y)

In [389]:
dragonnet = train_dragon(treatment.values.reshape(-1,1),
                                      y.values.reshape(-1,1),
                                      X.values,
                                      dragon='dragonnet',
                                      targeted_regularization=True,
                                      val_split=0.2, neurons_per_layer=100, batch_size=64, epochs=100,
                                      verbose=True)

I am here making dragonnet
Train on 597 samples, validate on 150 samples
Epoch 1/100
597/597 [==============================] - 1s 1ms/step - loss: 2841.0125 - regression_loss: 1321.2487 - binary_classification_loss: 40.0279 - treatment_accuracy: 0.7780 - track_epsilon: 0.0148 - val_loss: 2121.3647 - val_regression_loss: 827.0235 - val_binary_classification_loss: 33.0737 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0195
Epoch 2/100
597/597 [==============================] - 0s 61us/step - loss: 763.6118 - regression_loss: 338.6924 - binary_classification_loss: 32.3364 - treatment_accuracy: 0.8593 - track_epsilon: 0.0209 - val_loss: 347.3359 - val_regression_loss: 128.6071 - val_binary_classification_loss: 33.6181 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0205
Epoch 3/100
597/597 [==============================] - 0s 78us/step - loss: 301.3724 - regression_loss: 130.7361 - binary_classification_loss: 27.8279 - treatment_accuracy: 0.8433 - track_epsilon: 0.0196 -

Epoch 37/300
597/597 [==============================] - 0s 49us/step - loss: 154.6685 - regression_loss: 60.3431 - binary_classification_loss: 25.2238 - treatment_accuracy: 0.8433 - track_epsilon: 0.0054 - val_loss: 189.8671 - val_regression_loss: 61.3494 - val_binary_classification_loss: 36.9981 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0025
Epoch 38/300
597/597 [==============================] - 0s 49us/step - loss: 155.2768 - regression_loss: 60.3209 - binary_classification_loss: 25.2121 - treatment_accuracy: 0.8433 - track_epsilon: 0.0014 - val_loss: 196.2463 - val_regression_loss: 64.2867 - val_binary_classification_loss: 37.2283 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0039
Epoch 39/300
597/597 [==============================] - 0s 53us/step - loss: 153.5994 - regression_loss: 59.8586 - binary_classification_loss: 25.1677 - treatment_accuracy: 0.8593 - track_epsilon: 0.0045 - val_loss: 193.8690 - val_regression_loss: 63.0034 - val_binary_classificatio

597/597 [==============================] - 0s 47us/step - loss: 148.1820 - regression_loss: 56.9892 - binary_classification_loss: 24.8085 - treatment_accuracy: 0.8561 - track_epsilon: 0.0013 - val_loss: 193.7387 - val_regression_loss: 63.7617 - val_binary_classification_loss: 36.4481 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0013

Epoch 00081: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.
Epoch 82/300
597/597 [==============================] - 0s 48us/step - loss: 147.5900 - regression_loss: 56.6485 - binary_classification_loss: 24.8080 - treatment_accuracy: 0.8497 - track_epsilon: 0.0014 - val_loss: 192.8546 - val_regression_loss: 63.3035 - val_binary_classification_loss: 36.4506 - val_treatment_accuracy: 0.7244 - val_track_epsilon: 0.0014
Epoch 83/300
597/597 [==============================] - 0s 51us/step - loss: 146.3882 - regression_loss: 56.5048 - binary_classification_loss: 24.8048 - treatment_accuracy: 0.8465 - track_epsilon: 0.0013 - val_loss

In [390]:
dragonnet_preds = dragonnet.predict(X.values)
dragonnet_ite = (dragonnet_preds[:,1] - dragonnet_preds[:,0]).reshape(-1,1)
dragonnet_ate = dragonnet_ite.mean()

In [391]:
mae = lambda true,pred: np.mean(np.abs(true - pred))

In [392]:
df_preds = pd.DataFrame([s_ite.ravel(),
                          t_ite.ravel(),
                          x_ite.ravel(),
                          r_ite.ravel(),
                          dragonnet_ite.ravel(),
                          tau.ravel(),
                          treatment.ravel(),
                          y.ravel()],
                       index=['S','T','X','R','dragonnet','tau','w','y']).T

df_cumgain = get_cumgain(df_preds)

In [409]:
df_result = pd.DataFrame([s_ate, t_ate, x_ate, r_ate, dragonnet_ate, tau.mean()],
                     index=['S','T','X','R','dragonnet','actual'], columns=['ATE'])
df_result['MAE'] = [mae(t,p) for t,p in zip([s_ite, t_ite, x_ite, r_ite, dragonnet_ite],
                                         [tau.values.reshape(-1,1)]*5 )
                ] + [None]
df_result['AUUC'] = auuc_score(df_preds)

### Loop through all files

In [84]:
df_result_all = pd.DataFrame()

for filename in sorted(os.listdir(ihdp_dir)):
    df = pd.read_csv(f'{ihdp_dir}/{filename}', header=None)
    col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ]

    for i in range(1,26):
        col.append("x"+str(i))
    df.columns = col

    X = df.loc[:,'x1':]
    treatment = df['treatment']
    y = df['y_factual']
    tau = df.apply(lambda d: d['y_factual'] - d['y_cfactual']
                   if d['treatment']==1
                   else d['y_cfactual'] - d['y_factual'], axis=1)

    p_model = LogisticRegressionCV(penalty='elasticnet', solver='saga', l1_ratios=np.linspace(0,1,4),
                                   cv=StratifiedKFold(n_splits=3, shuffle=False))
    p_model.fit(X, treatment)
    p = p_model.predict_proba(X)[:, 1]

    s_learner = BaseSRegressor(XGBRegressor())
    s_ate = s_learner.estimate_ate(X, treatment, y)[0]
    s_ite = s_learner.fit_predict(X, treatment, y)

    t_learner = BaseTRegressor(XGBRegressor())
    t_ate = t_learner.estimate_ate(X, treatment, y)[0][0]
    t_ite = t_learner.fit_predict(X, treatment, y)

    x_learner = BaseXRegressor(XGBRegressor())
    x_ate = x_learner.estimate_ate(X, p, treatment, y)[0][0]
    x_ite = x_learner.fit_predict(X, p, treatment, y)

    r_learner = BaseRRegressor(XGBRegressor())
    r_ate = r_learner.estimate_ate(X, p, treatment, y)[0][0]
    r_ite = r_learner.fit_predict(X, p, treatment, y)

    dragonnet = train_dragon(treatment.values.reshape(-1,1),
                                          y.values.reshape(-1,1),
                                          X.values,
                                          dragon='dragonnet',
                                          targeted_regularization=True,
                                          val_split=0.2, neurons_per_layer=100, batch_size=64, epochs=100,
                                          verbose=False)

    dragonnet_preds = dragonnet.predict(X.values)
    dragonnet_ite = (dragonnet_preds[:,1] - dragonnet_preds[:,0]).reshape(-1,1)
    dragonnet_ate = dragonnet_ite.mean()

    mae = lambda true,pred: np.mean(np.abs(true - pred))

    df_preds = pd.DataFrame([s_ite.ravel(),
                              t_ite.ravel(),
                              x_ite.ravel(),
                              r_ite.ravel(),
                              dragonnet_ite.ravel(),
                              tau.ravel(),
                              treatment.ravel(),
                              y.ravel()],
                           index=['S','T','X','R','dragonnet','tau','w','y']).T

    df_cumgain = get_cumgain(df_preds)

    df_result = pd.DataFrame([s_ate, t_ate, x_ate, r_ate, dragonnet_ate, tau.mean()],
                         index=['S','T','X','R','dragonnet','actual'], columns=['ATE'])
    df_result['MAE'] = [mae(t,p) for t,p in zip([s_ite, t_ite, x_ite, r_ite, dragonnet_ite],
                                             [tau.values.reshape(-1,1)]*5 )
                    ] + [None]
    df_result['AUUC'] = auuc_score(df_preds)
    df_result.reset_index(inplace=True)
    df_result['file'] = filename
    df_result_all = pd.concat((df_result_all, df_result))
    
    print(f'Done with {filename}')

df_result_all.rename({'index':'model'}, axis=1, inplace=True)

# manual adjustment
df_result_all.loc[df_result_all.apply(lambda d: d['model']=='dragonnet' and np.isnan(d['MAE']), axis=1),
                 'AUUC'] = np.nan

Done with ihdp_npci_1.csv
Done with ihdp_npci_10.csv
Done with ihdp_npci_11.csv
Done with ihdp_npci_12.csv
Batch 6: Invalid loss, terminating training
Done with ihdp_npci_13.csv
Done with ihdp_npci_14.csv
Done with ihdp_npci_15.csv
Done with ihdp_npci_16.csv
Done with ihdp_npci_17.csv
Done with ihdp_npci_18.csv
Done with ihdp_npci_19.csv
Done with ihdp_npci_2.csv
Done with ihdp_npci_20.csv
Batch 0: Invalid loss, terminating training
Done with ihdp_npci_21.csv
Done with ihdp_npci_22.csv
Done with ihdp_npci_23.csv
Done with ihdp_npci_24.csv
Done with ihdp_npci_25.csv
Batch 9: Invalid loss, terminating training
Batch 0: Invalid loss, terminating training
Done with ihdp_npci_26.csv
Done with ihdp_npci_27.csv
Batch 6: Invalid loss, terminating training
Done with ihdp_npci_28.csv
Done with ihdp_npci_29.csv
Done with ihdp_npci_3.csv
Done with ihdp_npci_30.csv
Done with ihdp_npci_31.csv
Done with ihdp_npci_32.csv
Done with ihdp_npci_33.csv
Batch 6: Invalid loss, terminating training
Done with 

In [96]:
df_result_all.groupby(['file','model']).mean()

ATE       MAE      AUUC
file            model                                   
ihdp_npci_1.csv R           3.392378  1.379784  0.539915
                S           3.887754  1.108337  0.545272
                T           3.949531  1.087281  0.551643
                X           4.010829  1.130968  0.543934
                actual      4.029661       NaN       NaN
...                              ...       ...       ...
ihdp_npci_9.csv S          10.467238  8.312217  1.147244
                T          10.437422  3.475345  1.166413
                X           9.019373  6.895362  1.145507
                actual     10.460491       NaN       NaN
                dragonnet        NaN       NaN       NaN

[300 rows x 3 columns]

In [93]:
df_result_all.groupby('model').mean()[['MAE','AUUC']]

,MAE,AUUC
model,,
R,2.432765,0.755999
S,2.599897,0.763107
T,1.547712,0.777792
X,2.188928,0.765631
actual,NaN,NaN
dragonnet,1.219804,0.673524


### Compare performance with result from simulation script

In [9]:
import pickle

In [80]:
file_idx = 42

with np.load(f'/Users/mike.yung/Documents/repos/dragonnet/result/ihdp/dragonnet/{file_idx}/baseline/0_replication_train.npz') as f:
    _pred = f['q_t1'] - f['q_t0']
    _idx = f['index']

with np.load(f'/Users/mike.yung/Documents/repos/dragonnet/result/ihdp/dragonnet/{file_idx}/simulation_outputs.npz') as f:
    _t = f['t']
    _y = f['y']
    _y_cf = f['y_cf']    

df_sim = pd.DataFrame({'y':_y.ravel(),
                       'y_cf': _y_cf.ravel(),
                       't': _t.ravel(),
})
df_sim['tau'] = df_sim.apply(lambda d: d['y'] - d['y_cf'] if d['t']==1 else d['y_cf'] - d['y'], axis=1)    

print(mae(df_sim.loc[_idx, 'tau'], _pred))
df_result_all[df_result_all['file']==f'ihdp_npci_{file_idx+1}.csv']

1.2460000174576988


,model,ATE,MAE,AUUC,file
0,S,4.417201,1.239931,0.583065,ihdp_npci_43.csv
1,T,4.618019,1.088651,0.594821,ihdp_npci_43.csv
2,X,4.415116,1.181548,0.586271,ihdp_npci_43.csv
3,R,3.898648,1.490834,0.572499,ihdp_npci_43.csv
4,dragonnet,4.394847,1.128743,0.591419,ihdp_npci_43.csv
5,actual,4.454654,NaN,NaN,ihdp_npci_43.csv
